In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from math import sqrt

#загрузка датасета
ratings = pd.read_csv('user_ratings.csv')
user_ratings = pd.read_csv('sample_submission.csv')
user_ratings_Id_list = user_ratings["Id"].tolist()

#выпиливаем ненужные столбцы
un_ratings = ["watching_status", "watched_episodes"]
ratings = ratings.drop(un_ratings, 1)

#счетчик и массив с предсказаниями
count = 0
ratings_predict = []

#обход всех пользователей из sample_submission.csv
for user_rating in user_ratings_Id_list:
  user, anime = user_rating.split(" ")
  user = int(user)
  anime = int(anime)
  print("user_id {}".format(user), end=" ")
  print("anime_id {}".format(anime))
  ratings = ratings.loc[ratings['rating'] != 0]
  #выбираем пользователя для рекомендации и делим его просмотренные тайтлы на тестовые и валидационные
  inputMovies = ratings.where(ratings["user_id"] == user).dropna()
  #если нет оценок
  if inputMovies.empty:
    print(0)
    ratings_predict.append(0)
    count+=1
    print(count, end=" ")
    print(0)
    continue
  #убираем отметки пользователя ...
  ratings_test = ratings.loc[ratings['user_id'] != user]

  userSubset = ratings_test[ratings_test["anime_id"].isin(inputMovies['anime_id'].tolist())]
  userSubsetGroup = userSubset.groupby(['user_id'])
  userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)
  userSubsetGroup = userSubsetGroup[0:100]
  
  pearsonCorrelationDict = {}

  #Pearson Correlation
  for name, group in userSubsetGroup:
      group = group.sort_values(by='anime_id')
      inputMovies = inputMovies.sort_values(by='anime_id')
      nRatings = len(group)
      temp_df = inputMovies[inputMovies['anime_id'].isin(group['anime_id'].tolist())]
      tempRatingList = temp_df['rating'].tolist()
      tempGroupList = group['rating'].tolist()

      Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
      Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
      Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
      
      if Sxx != 0 and Syy != 0:
          pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
      else:
          pearsonCorrelationDict[name] = 0
      
  pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
  pearsonDF.columns = ['similarityIndex']
  pearsonDF['user_id'] = pearsonDF.index
  pearsonDF.index = range(len(pearsonDF))

  topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:100]

  topUsersRating=topUsers.merge(ratings_test, left_on='user_id', right_on='user_id', how='inner')

  topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']

  tempTopUsersRating = topUsersRating.groupby('anime_id').sum()[['similarityIndex','weightedRating']]
  tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']

  #из предсказанных оценок выбираем по Id anime
  recommendation_df = pd.DataFrame()
  recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
  recommendation_df['anime_id'] = tempTopUsersRating.index
  recommendation_df = recommendation_df.rename_axis(None)
  recommendation_df_ped = recommendation_df.loc[recommendation_df['anime_id'] == anime]
  predict_value = recommendation_df_ped['weighted average recommendation score'].tolist()
  print(predict_value)
  #схраняем предсказание
  ratings_predict.append(predict_value[0])
  count+=1
  print(count, end=" ")
  print(predict_value[0])

#удаление строки с рандомными предсказаниями и добалением наших
#генерация нового csv
dict_buffer = {'rating', ratings_predict}
sample_submission_new = pd.DataFrame(user_ratings['Id'], dict_buffer)
#сохранение csv
sample_submission_new.to_csv('./sample_submission_new')

user_id 0 anime_id 249
[8.466423028587334]
1 8.466423028587334
user_id 0 anime_id 4181
[8.897027834622182]
2 8.897027834622182
user_id 0 anime_id 813
[7.790536705728977]
3 7.790536705728977
user_id 0 anime_id 2167
[8.345107855276785]
4 8.345107855276785
user_id 1 anime_id 1838
[4.31575323603378]
5 4.31575323603378
user_id 1 anime_id 28223
[7.990095870159574]
6 7.990095870159574
user_id 1 anime_id 14813
[7.332210234005343]
7 7.332210234005343
user_id 1 anime_id 530
[7.188218880288332]
8 7.188218880288332
user_id 1 anime_id 10152
[6.982703299486075]
9 6.982703299486075
user_id 1 anime_id 9656
[7.8526059917042605]
10 7.8526059917042605
user_id 1 anime_id 2923
[6.602630537380138]
11 6.602630537380138
user_id 1 anime_id 32729
[6.809158003715116]
12 6.809158003715116
user_id 1 anime_id 1222
[6.636357138415158]
13 6.636357138415158
user_id 1 anime_id 317
[7.364196912352745]
14 7.364196912352745
user_id 2 anime_id 6702
0
15 0
user_id 2 anime_id 20
0
16 0
user_id 2 anime_id 1482
0
17 0
user_id 

KeyboardInterrupt: ignored